In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- Load prediction results from Excel ---
df_pred = pd.read_excel("single_rlstm_predictions.xlsx", sheet_name="Predicted Energy").drop(columns=["Week"])
df_std = pd.read_excel("single_rlstm_predictions.xlsx", sheet_name="Uncertainty (StdDev)").drop(columns=["Week"])
df_true = pd.read_excel("single_rlstm_predictions.xlsx", sheet_name="True Energy").drop(columns=["Week"])
df_time = pd.read_excel("X_test_scaled.xlsx")

# --- Reconstruct flattened arrays ---
mu_flat = df_pred.to_numpy().reshape(-1)
std_flat = df_std.to_numpy().reshape(-1)
y_true_flat = df_true.to_numpy().reshape(-1)

# --- Time index (rebuild 168-hour weekly time series) ---
output_window = 168
expanded_time = pd.date_range(
    start=pd.to_datetime(df_time["datetime"].iloc[output_window]),
    periods=len(mu_flat),
    freq='H'
)

# --- Plotting ---
fig = go.Figure()

# 🔵 True Energy
fig.add_trace(go.Scatter(
    x=expanded_time,
    y=y_true_flat,
    mode='lines',
    name='True Energy',
    line=dict(color='blue')
))

# ⚫ Predicted Mean
fig.add_trace(go.Scatter(
    x=expanded_time,
    y=mu_flat,
    mode='lines',
    name='Predicted Mean',
    line=dict(color='black')
))

# 🟠 Uncertainty Band (±2 std)
fig.add_trace(go.Scatter(
    x=np.concatenate([expanded_time, expanded_time[::-1]]),
    y=np.concatenate([mu_flat - 2 * std_flat, (mu_flat + 2 * std_flat)[::-1]]),
    fill='toself',
    fillcolor='rgba(255,165,0,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='±2 Std Dev (Uncertainty)'
))

# ❌ Outliers
lower = mu_flat - 2 * std_flat
upper = mu_flat + 2 * std_flat
outliers = (y_true_flat < lower) | (y_true_flat > upper)

fig.add_trace(go.Scatter(
    x=expanded_time[outliers],
    y=y_true_flat[outliers],
    mode='markers',
    name='Outside 95% CI',
    marker=dict(color='red', symbol='x', size=7),
    showlegend=True
))

# Layout
fig.update_layout(
    title='📊 Single RLSTM Forecast with Uncertainty (All Weeks)',
    xaxis_title='Datetime',
    yaxis_title='Energy',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    template='plotly_white',
    height=600
)

# Save as HTML
fig.write_html("single_rlstm_uncertainty_all_weeks.html")
print("✅ Saved: single_rlstm_uncertainty_all_weeks.html")


C:\Users\arman\AppData\Local\Temp\ipykernel_13076\172809729.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expanded_time = pd.date_range(


✅ Saved: single_rlstm_uncertainty_all_weeks.html


In [4]:
coverage = 100 * np.mean(~outliers)
print(f"✅ Coverage within ±2 Std Dev: {coverage:.2f}%")


✅ Coverage within ±2 Std Dev: 87.94%


In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- Load prediction results from Ensemble RLSTM ---
df_pred = pd.read_excel("final_rlstm_predictions.xlsx", sheet_name="Predicted Energy").drop(columns=["Week"])
df_std = pd.read_excel("final_rlstm_predictions.xlsx", sheet_name="Uncertainty (StdDev)").drop(columns=["Week"])
df_true = pd.read_excel("final_rlstm_predictions.xlsx", sheet_name="True Energy").drop(columns=["Week"])
df_time = pd.read_excel("X_test_scaled.xlsx")

# --- Reconstruct flattened arrays ---
mu_flat = df_pred.to_numpy().reshape(-1)
std_flat = df_std.to_numpy().reshape(-1)
y_true_flat = df_true.to_numpy().reshape(-1)

# --- Time index (rebuild 168-hour weekly time series) ---
output_window = 168
expanded_time = pd.date_range(
    start=pd.to_datetime(df_time["datetime"].iloc[output_window]),
    periods=len(mu_flat),
    freq='H'
)

# --- Calculate outliers (outside ±2 std) ---
lower = mu_flat - 2 * std_flat
upper = mu_flat + 2 * std_flat
outliers = (y_true_flat < lower) | (y_true_flat > upper)

# ✅ Calculate coverage within ±2 std
coverage = 100 * np.mean(~outliers)
print(f"✅ Coverage within ±2 Std Dev: {coverage:.2f}%")

# --- Plotting ---
fig = go.Figure()

# 🔵 True Energy
fig.add_trace(go.Scatter(
    x=expanded_time,
    y=y_true_flat,
    mode='lines',
    name='True Energy',
    line=dict(color='blue')
))

# ⚫ Predicted Mean
fig.add_trace(go.Scatter(
    x=expanded_time,
    y=mu_flat,
    mode='lines',
    name='Predicted Mean',
    line=dict(color='black')
))

# 🟠 Uncertainty Band (±2 std)
fig.add_trace(go.Scatter(
    x=np.concatenate([expanded_time, expanded_time[::-1]]),
    y=np.concatenate([mu_flat - 2 * std_flat, (mu_flat + 2 * std_flat)[::-1]]),
    fill='toself',
    fillcolor='rgba(255,165,0,0.3)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='±2 Std Dev (Uncertainty)'
))

# ❌ Outliers
fig.add_trace(go.Scatter(
    x=expanded_time[outliers],
    y=y_true_flat[outliers],
    mode='markers',
    name='Outside 95% CI',
    marker=dict(color='red', symbol='x', size=7),
    showlegend=True
))

# Layout
fig.update_layout(
    title=f'📊 Ensemble RLSTM Forecast with Uncertainty — Coverage: {coverage:.2f}%',
    xaxis_title='Datetime',
    yaxis_title='Energy',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    template='plotly_white',
    height=600
)

# Save to HTML
fig.write_html("ensemble_rlstm_uncertainty_all_weeks.html")
print("✅ Saved: ensemble_rlstm_uncertainty_all_weeks.html")


C:\Users\arman\AppData\Local\Temp\ipykernel_13076\1795277378.py:18: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



✅ Coverage within ±2 Std Dev: 96.05%
✅ Saved: ensemble_rlstm_uncertainty_all_weeks.html
